# PromptTemplate 튜토리얼

## 개요
**PromptTemplate** 은 AI 모델에 전달할 프롬프트를 **변수화된 템플릿** 형태로 정의·재사용하기 위한 도구입니다. 반복 작업을 줄이고, 형식을 표준화하며, 협업과 유지보수를 쉽게 합니다.

## 학습 목표
- **PromptTemplate 기본**: 변수 기반 템플릿 작성과 포맷팅
- **파일 기반 템플릿 관리**: YAML로 프롬프트를 분리·버전관리
- **ChatPromptTemplate**: 역할 기반 대화 프롬프트 구성
- **MessagePlaceholder**: 대화 이력(가변 길이) 삽입

## 왜 사용하는가
| 장점 | 설명 |
|---|---|
| **재사용성** | 한 번 정의한 템플릿을 여러 입력에 반복 사용 |
| **일관성** | 팀/서비스 전반에서 동일한 질의 형식 유지 |
| **효율성** | 매번 문장을 새로 작성하지 않아 작업 시간 절감 |
| **안정성** | 검증된 프롬프트 패턴을 표준으로 채택 |

## 환경 설정
- `.env` 파일을 사용해 API 키를 관리합니다.
- LangSmith 트레이싱을 프로젝트 수준으로 활성화합니다(선택).
- OpenRouter 경유로 ChatOpenAI를 사용합니다.

In [ ]:
# API KEY 구성 및 LangSmith 설정
from dotenv import load_dotenv

# .env 로드
load_dotenv(override=True)

In [ ]:
# Set LangSmith trace https://smith.langchain.com
# Add LANGCHAIN_API_KEY in .env file
from langchain_teddynote import logging

# 프로젝트명 설정
logging.langsmith("LangChain-Tutorial")

## PromptTemplate 기본

### LLM 모델 준비
PromptTemplate을 사용하려면 우선 대화 모델(LLM)을 구성합니다. 본 튜토리얼은 OpenRouter를 통해 OpenAI 계열 모델을 호출합니다.

In [ ]:
# OpenRouter 경유로 ChatOpenAI 구성
import os
from langchain_openai import ChatOpenAI

# Create ChatOpenAI object
llm = ChatOpenAI(
    temperature=0.1,
    model="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

## PromptTemplate 기본 사용법

### 핵심 문법

- 변수 표기: `{변수명}`

- 템플릿 생성: `PromptTemplate.from_template(...)`
- 값 대입: `format(...)` 또는 체인에서 매핑 입력

### 예시

일반적인 방법:
```
"대한민국의 수도는 어디인가요?"
"일본의 수도는 어디인가요?" 
"프랑스의 수도는 어디인가요?"
```

PromptTemplate 방법:
```
"{country}의 수도는 어디인가요?"
```

템플릿을 사용하면 동일한 형식을 유지하면서 여러 입력값에 쉽게 재사용할 수 있습니다.

In [ ]:
# PromptTemplate 클래스를 불러옵니다
from langchain_core.prompts import PromptTemplate

# 템플릿 문자열을 정의합니다. {country}는 나중에 값이 들어갈 변수입니다
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 사용하여 PromptTemplate 객체를 생성합니다
prompt = PromptTemplate.from_template(template)
prompt

### 변수에 값 대입하기
`{country}` 변수에 실제 값을 넣어 최종 프롬프트를 생성합니다.

In [ ]:
# format 메소드를 사용하여 {country} 변수에 "대한민국" 값을 대입합니다
prompt = prompt.format(country="대한민국")
prompt

In [ ]:
# 새로운 템플릿을 정의합니다
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 생성합니다
prompt = PromptTemplate.from_template(template)

# 파이프라인(체인)을 생성합니다: 프롬프트 → LLM
chain = prompt | llm

In [ ]:
# 체인을 실행합니다: "대한민국"이 {country} 변수에 자동으로 대입됩니다
result = chain.invoke({"country": "대한민국"})
print(result.content)

## 파일 기반 템플릿 관리

### 왜 파일로 관리하나

코드와 프롬프트를 분리하면 협업과 변경 관리가 쉬워집니다.

### 활용상의 장점

실제 프로덕션 환경에서는 다음과 같은 이유로 파일 기반 관리를 선호합니다:

- **협업**: 비개발자도 YAML만 수정하여 참여 가능
- **버전 관리**: Git으로 변경 이력 추적 용이
- **다국어/재사용**: 언어·프로젝트별 템플릿 분리
- **유지보수**: 코드 수정 없이 프롬프트 개선 가능

## YAML 예시

**YAML** 은 사람이 읽고 쓰기 쉬운 데이터 형식입니다:

```yaml
_type: prompt
input_variables: ["fruit"]
template: "What color is {fruit}?"
```



In [ ]:
# load_prompt 함수를 불러옵니다
from langchain_core.prompts import load_prompt

# YAML 파일에서 프롬프트 템플릿을 로드합니다
prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

In [ ]:
# 로드된 템플릿에 값을 대입해서 결과를 확인합니다
prompt.format(fruit="사과")

In [ ]:
# 다른 YAML 파일도 로드해봅시다
prompt2 = load_prompt("prompts/capital.yaml")

# 국가 변수에 값을 대입하여 결과를 출력합니다
print(prompt2.format(country="대한민국"))

## ChatPromptTemplate

### ChatPromptTemplate이 뭐가 다를까?

일반적인 `PromptTemplate` 은 **단순한 텍스트 질문** 에 적합합니다. 하지만 **ChatGPT 같은 대화형 AI** 와 소통할 때는 **ChatPromptTemplate** 이 훨씬 효과적입니다.

#### 일반 PromptTemplate
```
"안녕하세요. 대한민국의 수도가 어디인지 알려주세요."
```

#### ChatPromptTemplate
```
시스템: "당신은 친절한 지리 전문가입니다."
사용자: "안녕하세요!"
AI: "안녕하세요! 지리에 관해 무엇이든 물어보세요."
사용자: "대한민국의 수도는 어디인가요?"
```

### ChatPromptTemplate의 특별한 기능

- **역할 설정**: 시스템 메시지로 AI의 성격과 역할 정의
- **대화 맥락**: 이전 대화 내용을 기억하고 참조
- **연속성**: 자연스러운 대화 흐름 유지
- **다양한 메시지 타입**: system, human, ai 역할 구분

In [ ]:
# ChatPromptTemplate 클래스를 불러옵니다
from langchain_core.prompts import ChatPromptTemplate

# 간단한 채팅 프롬프트를 생성합니다 (기본적으로 human 역할)
chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

In [ ]:
# 변수에 값을 대입하여 최종 프롬프트를 생성합니다
chat_prompt.format(country="대한민국")

### 다중 메시지로 맥락 구성

역할과 순서를 지정한 메시지 리스트로 실제 대화 상황을 구성하고, 변수로 사용자 입력을 주입할 수 있습니다. 

#### 구성 형태

ChatPromptTemplate은 "(역할, 메시지)" 튜플 목록으로 메시지를 정의합니다:

```
(역할, 대사) 형태의 튜플로 구성
("system", "당신은 친절한 AI 어시스턴트입니다")
("human", "안녕하세요!")  
("ai", "안녕하세요! 무엇을 도와드릴까요?")
```

In [ ]:
# ChatPromptTemplate을 다시 불러옵니다
from langchain_core.prompts import ChatPromptTemplate

# 여러 메시지로 구성된 대화 템플릿을 생성합니다
chat_template = ChatPromptTemplate.from_messages(
    [
        # (역할, 메시지) 형태의 튜플 리스트
        (
            "system",
            "You are a helpful AI assistant. Your name is {name}.",
        ),  # 시스템 설정
        ("human", "반가워요!"),  # 사용자 첫 인사
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),  # AI 응답
        ("human", "{user_input}"),  # 사용자의 실제 질문 (변수)
    ]
)

# format_messages 메소드로 변수들을 대입하여 완전한 대화를 생성합니다
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

### 생성된 메시지 실행

포맷팅된 메시지 목록을 그대로 LLM에 전달해 응답을 확인합니다.

In [ ]:
# 생성된 메시지들을 LLM에 직접 전달하여 답변을 받습니다
result = llm.invoke(messages)
print(result.content)

### 체인(Chain)으로 연결

프롬프트와 LLM을 파이프(`|`)로 연결하면 입력 매핑과 호출 과정을 단순화할 수 있습니다.

In [ ]:
# chat_template 프롬프트와 llm 모델을 연결하여 체인 생성
chain = chat_template | llm

In [ ]:
# 체인을 실행합니다: 변수들이 자동으로 대입되고 LLM이 답변을 생성합니다
result = chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"})
print(result.content)

In [ ]:
# 체인을 실행합니다: 변수들이 자동으로 대입되고 LLM이 답변을 생성합니다
chain = chat_template | llm
result = chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"})
print(result.content)

## MessagePlaceholder

### 필요성

대화 길이와 구성은 상황마다 달라집니다. **MessagePlaceholder** 는 가변 길이의 대화 이력을 한 번에 삽입할 수 있도록 해 줍니다.

### 특징

- **가변 길이**: 메시지 수와 내용이 유동적이어도 처리 가능
- **동적 삽입**: 실행 시점에 대화 배열을 전달
- **재사용성**: 템플릿 구조는 유지하고 맥락만 교체

### 활용 예
- **대화 요약**: 다양한 길이의 대화를 요약
- **챗봇**: 이전 대화 이력을 맥락으로 전달
- **업무 노트/회의록**: 회차마다 다른 참석자·내용 반영

In [ ]:
# 필요한 클래스들을 불러옵니다
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# MessagePlaceholder를 포함한 채팅 프롬프트를 생성합니다
chat_prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지: AI의 역할을 정의합니다
        (
            "system",
            "You are a summarization expert AI assistant. Your task is to summarize conversations using key keywords.",
        ),
        # 가변적인 대화 내용이 들어갈 자리를 만듭니다
        MessagesPlaceholder(variable_name="conversation"),
        # 마지막 인간 메시지: 요약 지시사항
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)

### MessagePlaceholder 동작 테스트

`conversation` 변수에 대화 메시지 리스트를 실행 시점에 전달합니다.

In [ ]:
# 최종 실행을 위한 체인을 생성합니다 (StrOutputParser 추가로 문자열 결과만 받기)
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# 체인을 실행하여 대화를 요약합니다
result = chain.invoke(
    {
        "word_count": 5,  # 5단어로 요약
        "conversation": [  # 요약할 대화 내용
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)
print(f"요약 결과: {result}")